# Australian Boreholes

Looking at the data from the [**Australian Borehole Stratigraphic Unit Compilation**](https://ecat.ga.gov.au/geonetwork/srv/eng/catalog.search#/metadata/147641) 2023 v1.0

In [18]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib as plt
import pygmt

In [2]:
absuc = pd.read_csv("data/ABSUC/ABSUC_v1_BASE.txt")  # preferred stratigraphic picks for the base of each unit
absuc.set_index("GA_GUID", inplace=True)
absuc.shape

/var/folders/t6/g_v51vw92p98wh69ncgrb0mh0000gn/T/ipykernel_69314/2604193656.py:1: DtypeWarning: Columns (4,28) have mixed types. Specify dtype option on import or set low_memory=False.
  absuc = pd.read_csv("data/ABSUC/ABSUC_v1_BASE.txt")  # preferred stratigraphic picks for the base of each unit


(468299, 28)

In [3]:
# remove entries with NaNs in the seemingly relevant depth measurements
absuc.dropna(
    axis=0,
    subset=("TD_MD_M", "TOP_MD_M", "BASE_MD_M", "TOP_AHD_M", "BASE_AHD_M"),
    inplace=True,
)

# remove some irrelevant columns
absuc.drop(
    labels=[
        "OBJECTID",
        "WD_AHD_M",
        "SRTM_HE_AHD_M",
        "DATA_SOURCE",
        "GA_HYDROSTRAT",
        "Z_M_SRTM",
        "COMMENT",
    ],
    axis=1,
    inplace=True
)


In [4]:
absuc[absuc["GA_BOREHOLE_GUID"] == "{412fc35f-d103-4b74-8ff7-751dbc86e18c}"]

,GA_BOREHOLE_GUID,GA_ENO,UWI,BOREHOLE_NAME,GDA94_LATITUDE,GDA94_LONGITUDE,GL_AHD_M,DATUM_ELEVATION_AHD_M,DATUM_NAME,TD_MD_M,...,GA_UNIT,GA_ASUD_NO,TOP_MD_M,BASE_MD_M,TOP_AHD_M,BASE_AHD_M,Z_AHD_M,HIERARCHY,TAG,PREFERRED
GA_GUID,,,,,,,,,,,,,,,,,,,,,
{bf221f26-842f-4783-b448-fe761ec8de62},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Namba Formation,13707.0,18.288,70.714,60.926,8.500,8.500,260,NaN,TB
{b86ba315-84ed-4e10-b07e-e93d6cb62216},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Eyre Formation,6451.0,70.714,148.742,8.500,-69.528,-69.528,260,NaN,TB
{ca89e0b9-d8a4-45af-b499-2c631a25c47b},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Winton Formation,20268.0,148.742,724.814,-69.528,-645.600,-645.600,260,NaN,TB
{22c9d65b-92c6-477e-bc9b-f40816efaf40},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Mackunda Formation,10952.0,724.814,816.254,-645.600,-737.040,-737.040,260,NaN,TB
{75c4f436-9094-4185-9460-45df00ad5e28},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Oodnadatta Formation,14592.0,816.254,1139.038,-737.040,-1059.824,-1059.824,260,NaN,TB
{dce51534-cc62-41a5-bb16-3d9913630a06},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Coorikiana Sandstone,26493.0,1139.038,1151.230,-1059.824,-1072.016,-1072.016,260,NaN,TB
{8621ba24-acd6-4c7e-a07b-00c5a4ed3e25},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Bulldog Shale,2891.0,1151.230,1418.844,-1072.016,-1339.630,-1339.630,260,NaN,TB
{c2b12d31-d4ce-4775-a1b9-6303d99725dd},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Cadna-owie Formation,3284.0,1418.844,1498.702,-1339.630,-1419.488,-1419.488,260,NaN,B
{69785e5c-2e63-43d4-9697-e7770c4a922d},{412fc35f-d103-4b74-8ff7-751dbc86e18c},10000.0,10000,380,-28.2179,140.86856,72.03,79.214,Kelly Bushing (KB),2740.762,...,Murta Formation,13577.0,1498.702,1536.192,-1419.488,-1456.978,-1456.978,260,NaN,B
